In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [12]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.now())
print(f"{current_date}")

2025-03-02 20:33:26.528658


In [13]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [14]:
from datetime import datetime, timedelta, timezone
current_date =current_date.floor("h")

In [15]:
current_date.to_datetime64()

numpy.datetime64('2025-03-02T20:00:00.000000000')

In [16]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=60)

In [17]:
fetch_data_to

Timestamp('2025-03-02 20:00:00')

In [18]:
fetch_data_from

Timestamp('2025-01-01 20:00:00')

In [19]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [20]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-01.
Loading data for 2024-01...
Total records: 2,964,624
Valid records: 2,911,483
Records dropped: 53,141 (1.79%)
Successfully processed data for 2024-01.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [21]:
rides

,pickup_datetime,pickup_location_id
15884,2025-01-02 05:29:47,2
2577207,2025-01-04 23:23:27,2
437063,2025-01-06 22:30:16,2
15992,2025-01-02 05:31:51,3
16164,2025-01-02 05:41:01,3
...,...,...
2972330,2025-03-02 19:58:10,263
2972677,2025-03-02 19:58:12,263
2972084,2025-03-02 19:58:15,263
2973487,2025-03-02 19:58:16,263


In [22]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [23]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-01-01 20:00:00,2,0
1,2025-01-01 21:00:00,2,0
2,2025-01-01 22:00:00,2,0
3,2025-01-01 23:00:00,2,0
4,2025-01-02 00:00:00,2,0
...,...,...,...
368635,2025-03-02 15:00:00,263,130
368636,2025-03-02 16:00:00,263,93
368637,2025-03-02 17:00:00,263,114
368638,2025-03-02 18:00:00,263,114


In [24]:
import pytz

est = pytz.timezone('US/Eastern')

ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(est, ambiguous="NaT", nonexistent="NaT")
ts_data = ts_data.dropna(subset=["pickup_hour"])
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-01-01 20:00:00-05:00,2,0
1,2025-01-01 21:00:00-05:00,2,0
2,2025-01-01 22:00:00-05:00,2,0
3,2025-01-01 23:00:00-05:00,2,0
4,2025-01-02 00:00:00-05:00,2,0
...,...,...,...
368635,2025-03-02 15:00:00-05:00,263,130
368636,2025-03-02 16:00:00-05:00,263,93
368637,2025-03-02 17:00:00-05:00,263,114
368638,2025-03-02 18:00:00-05:00,263,114


In [25]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368640 entries, 0 to 368639
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                     
---  ------              --------------   -----                     
 0   pickup_hour         368640 non-null  datetime64[ns, US/Eastern]
 1   pickup_location_id  368640 non-null  int16                     
 2   rides               368640 non-null  int16                     
dtypes: datetime64[ns, US/Eastern](1), int16(2)
memory usage: 4.2 MB


In [26]:
ts_data.tail()

,pickup_hour,pickup_location_id,rides
368635,2025-03-02 15:00:00-05:00,263,130
368636,2025-03-02 16:00:00-05:00,263,93
368637,2025-03-02 17:00:00-05:00,263,114
368638,2025-03-02 18:00:00-05:00,263,114
368639,2025-03-02 19:00:00-05:00,263,93


In [27]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')

# Initialize connection to Hopsworks
project = hopsworks.login(
    api_key_value=api_key,
    project=project_name
)
print(f"Successfully connected to Hopsworks project: {project_name}")

c:\Users\nolan\anaconda3\envs\sp25_taxi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-03-02 20:34:34,053 INFO: Initializing external client
2025-03-02 20:34:34,053 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:34:35,165 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Successfully connected to Hopsworks project: nolanphi


In [28]:
feature_store = project.get_feature_store()

In [29]:
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

In [30]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 368640/368640 | Elapsed Time: 00:25 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)